In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E0172018E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E017202A20>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage,SystemMessage
model.invoke([HumanMessage(content="Hi My name is Aakash and i am an AI engineer")])

AIMessage(content="Hi Aakash,\n\nIt's nice to meet you!  As an AI, I don't have the pleasure of meeting people in the real world, so it's always interesting to connect with someone who works in the field of AI.\n\nWhat kind of AI engineering do you specialize in?  Are you working on any exciting projects right now?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 22, 'total_tokens': 99, 'completion_time': 0.14, 'prompt_time': 0.002128625, 'queue_time': 0.250406673, 'total_time': 0.142128625}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--714f5a0a-b01c-40cc-943f-4f89e0845b85-0', usage_metadata={'input_tokens': 22, 'output_tokens': 77, 'total_tokens': 99})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi My name is Aakash and i am an AI engineer"),
        AIMessage(content="Hi Aakash, it's nice to meet you!\n\nThat's awesome that you're an AI engineer. What kind of projects are you working on?"),
        HumanMessage(content="Hey, whats my name and what do i do?")
    ]
)

AIMessage(content="You are Aakash, and you are an AI engineer.  \n\nIs there anything else you'd like to know or talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 78, 'total_tokens': 112, 'completion_time': 0.061818182, 'prompt_time': 0.005280868, 'queue_time': 0.27750575, 'total_time': 0.06709905}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--41fedb20-230e-4282-9181-1434fed4c64e-0', usage_metadata={'input_tokens': 78, 'output_tokens': 34, 'total_tokens': 112})

## MESSAGE HISTORY

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [8]:
response = with_message_history.invoke([HumanMessage(content="Hi My name is Aakash and i am an AI engineer")],
                            config=config)

In [9]:
response.content

"Hi Aakash, it's nice to meet you!\n\nThat's really cool! Being an AI engineer is fascinating.  \n\nWhat kind of AI projects are you working on? 🤖  I'm always eager to learn about what people are building in the field.\n"

In [10]:
# Change config --> change the session id
config1 = {"configurable":{"session_id":"chat2"}}
res = with_message_history.invoke([HumanMessage(content="whats my name")],config=config1)
res.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

### PROMPT TEMPLATES


In [11]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all questions to best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [12]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Aakash")]})

AIMessage(content="Hi Aakash, it's nice to meet you!  \n\nWhat can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 30, 'total_tokens': 59, 'completion_time': 0.052727273, 'prompt_time': 0.002334575, 'queue_time': 0.242335845, 'total_time': 0.055061848}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a47ec983-bdba-458b-a7a0-7e62b9bcc130-0', usage_metadata={'input_tokens': 30, 'output_tokens': 29, 'total_tokens': 59})

In [13]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [14]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke([HumanMessage(content="Hi My name is Aakash")],config = config)

response

AIMessage(content="Hello Aakash, it's nice to meet you!\n\nHow can I help you today?  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 30, 'total_tokens': 57, 'completion_time': 0.049090909, 'prompt_time': 0.002132145, 'queue_time': 0.251602824, 'total_time': 0.051223054}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d02ca7bc-ec7e-46ba-8a68-8ef81285267f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57})

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all questions to best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt|model

In [16]:
response = chain.invoke({"messages":[HumanMessage(content="Hi my name is Aakash")],"language":"Hindi"})
response.content

'नमस्ते आकाश!  😊 \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आपके कोई सवाल हैं? \n'

In [17]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [18]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke({"messages":[HumanMessage(content="Hi My name is Aakash")],
                                       "language":"hindi"},
                                       config = config)

response

AIMessage(content='नमस्ते अकाश! 👋  \n\nमुझे बहुत खुशी है कि आपने मुझसे बातचीत शुरू की!  😊 \n\nआपके सवालों के जवाब देने में मैं अपनी पूरी कोशिश करूँगा।  \n\nआप मुझे क्या पूछना चाहते हैं? 🤔 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 32, 'total_tokens': 103, 'completion_time': 0.129090909, 'prompt_time': 0.004617409, 'queue_time': 0.262637001, 'total_time': 0.133708318}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b978558b-357b-467b-9777-ea087eed6237-0', usage_metadata={'input_tokens': 32, 'output_tokens': 71, 'total_tokens': 103})

### MANAGE THE SIZE OF CONVERSATION HISTORY


In [24]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [25]:
messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="Hi i am aakash"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    
]

trimmer.invoke(messages)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [29]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

res = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem i asked?")],
    "language":"english",
    }
)
res.content

'You asked me "what\'s 2 + 2".  \n\n\n\nDo you want to try another one? 😄 \n'

In [30]:
# WRAP in message history

with_mesage_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable":{"session_id":"chat5"}}

In [31]:
response = with_message_history.invoke(
    {
        "messages":messages + [HumanMessage(content="Whats my name?")],
        "language":"English",
    },
    config = config
)

In [32]:
response.content

'Your name is Aakash.  😊'

### VECTOR STORES AND RETRIEVERS

In [33]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners,requiring relatively simple care",
        metadata={"source":"mammal-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech",
        metadata={"source":"mammal-pets-doc"},
    ),
]

In [34]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Goldfish are popular pets for beginners,requiring relatively simple care'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech')]

In [36]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
llm = ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001E024DEB950>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E024DA6120>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [37]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [38]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore

In [39]:
vectorstore.similarity_search("Cat")

[Document(id='11d18ef9-022e-4438-9114-e9354feab2f7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='13360e1a-d627-4071-853d-8724160669ca', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='780a29c8-b1fd-49b7-8d22-fa10cad3afd9', metadata={'source': 'mammal-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
 Document(id='185bc0c6-44ab-4243-8444-03d125b363a8', metadata={'source': 'mammal-pets-doc'}, page_content='Goldfish are popular pets for beginners,requiring relatively simple care')]

In [40]:
await vectorstore.asimilarity_search("Cat")

[Document(id='11d18ef9-022e-4438-9114-e9354feab2f7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(id='13360e1a-d627-4071-853d-8724160669ca', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
 Document(id='780a29c8-b1fd-49b7-8d22-fa10cad3afd9', metadata={'source': 'mammal-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
 Document(id='185bc0c6-44ab-4243-8444-03d125b363a8', metadata={'source': 'mammal-pets-doc'}, page_content='Goldfish are popular pets for beginners,requiring relatively simple care')]

In [42]:
vectorstore.similarity_search_with_score("Cat")

[(Document(id='11d18ef9-022e-4438-9114-e9354feab2f7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  0.9436445236206055),
 (Document(id='13360e1a-d627-4071-853d-8724160669ca', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness'),
  1.5413694381713867),
 (Document(id='780a29c8-b1fd-49b7-8d22-fa10cad3afd9', metadata={'source': 'mammal-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
  1.6284856796264648),
 (Document(id='185bc0c6-44ab-4243-8444-03d125b363a8', metadata={'source': 'mammal-pets-doc'}, page_content='Goldfish are popular pets for beginners,requiring relatively simple care'),
  1.7496695518493652)]

### Retrievers

#### TECHNIQUE 1

In [44]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["Cat","Dog"])

[[Document(id='11d18ef9-022e-4438-9114-e9354feab2f7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='13360e1a-d627-4071-853d-8724160669ca', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

#### TECHNIQUE 2

In [45]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1},
)
retriever.batch(["Cat","Dog"])

[[Document(id='11d18ef9-022e-4438-9114-e9354feab2f7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(id='13360e1a-d627-4071-853d-8724160669ca', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness')]]

In [46]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.
{question}
Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])
rag_chain = {"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response = rag_chain.invoke("tell me about dogs")
response.content

'According to the provided context, dogs are great companions, known for their loyalty and friendliness.'